In [1]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata
import pandas as pd
import numpy as np

C:\Users\User\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


# T1

From : sleepDay_merged.csv, dailyActivity_merged.csv, weightLogInfo_merged.csv, heartrate_seconds_merged.csv (aggregated daily if you can)

Features: TotalMinutesAsleep, TotalTimeInBed, Calories, TotalSteps, VeryActiveMinutes, SedentaryMinutes, BMI, AvgHeartRate
Label: SleepEfficiency

In [3]:
sleepDay_merged = pd.read_csv("Kaggle_Dataset/sleepDay_merged.csv") #Contains TotalMinutesAsleep, TotalTimeInBed
dailyActivity_merged = pd.read_csv("Kaggle_Dataset/dailyActivity_merged.csv") #Contains Calories, TotalSteps, VeryActiveMinutes, SedentaryMinutes
weightLogInfo_merged = pd.read_csv("Kaggle_Dataset/weightLogInfo_merged.csv")  # Contains BMI
heartrate_seconds_merged = pd.read_csv("Kaggle_Dataset/heartrate_seconds_merged.csv")

In [4]:
heartrate_seconds_merged['Time'] = pd.to_datetime(heartrate_seconds_merged['Time'])
heartrate_seconds_merged['Date'] = heartrate_seconds_merged['Time'].dt.date
daily_avg_heartrate = heartrate_seconds_merged.groupby(['Id', 'Date'])['Value'].mean().reset_index()
daily_avg_heartrate.rename(columns={'Value': 'AvgHeartRate'}, inplace=True)
daily_avg_heartrate  # Contains AvgHeartRate

,Id,Date,AvgHeartRate
0,2022484408,2016-04-12,75.804177
1,2022484408,2016-04-13,80.337584
2,2022484408,2016-04-14,72.628597
3,2022484408,2016-04-15,80.437382
4,2022484408,2016-04-16,75.960547
...,...,...,...
329,8877689391,2016-05-08,72.550523
330,8877689391,2016-05-09,89.615738
331,8877689391,2016-05-10,71.544377
332,8877689391,2016-05-11,89.149122


In [5]:
sleepDay_merged['SleepDay'] = pd.to_datetime(sleepDay_merged['SleepDay']).dt.date
dailyActivity_merged['ActivityDate'] = pd.to_datetime(dailyActivity_merged['ActivityDate']).dt.date
weightLogInfo_merged['Date'] = pd.to_datetime(weightLogInfo_merged['Date']).dt.date

sleepDay_merged.rename(columns={'SleepDay': 'Date'}, inplace=True)
dailyActivity_merged.rename(columns={'ActivityDate': 'Date'}, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_39676\1983187807.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weightLogInfo_merged['Date'] = pd.to_datetime(weightLogInfo_merged['Date']).dt.date


In [6]:
T1_sleep = sleepDay_merged[['Id', 'Date', 'TotalMinutesAsleep', 'TotalTimeInBed']]
T1_activity = dailyActivity_merged[['Id', 'Date', 'Calories', 'TotalSteps', 'VeryActiveMinutes', 'SedentaryMinutes']]
T1_weight = weightLogInfo_merged[['Id', 'Date', 'BMI']]
T1_heartrate = daily_avg_heartrate[['Id', 'Date', 'AvgHeartRate']]

T1_merged_df = pd.merge(T1_sleep, T1_activity, on=['Id', 'Date'], how='outer')
T1_merged_df = pd.merge(T1_merged_df, T1_weight, on=['Id', 'Date'], how='outer')
T1_merged_df = pd.merge(T1_merged_df, T1_heartrate, on=['Id', 'Date'], how='outer')

T1_merged_df.dropna(how='all', subset=[
    'TotalMinutesAsleep', 'TotalTimeInBed', 'Calories', 'TotalSteps',
    'VeryActiveMinutes', 'SedentaryMinutes', 'BMI', 'AvgHeartRate'
], inplace=True)

T1_merged_df['SleepEfficiency'] = T1_merged_df['TotalMinutesAsleep'] / T1_merged_df['TotalTimeInBed']
T1_merged_df['SleepEfficiency'] = T1_merged_df['SleepEfficiency'].replace([np.inf, -np.inf], np.nan)
T1_merged_df = T1_merged_df[~((T1_merged_df['SleepEfficiency'] == 0) | (T1_merged_df['SleepEfficiency'].isna()))]

T1_merged_df

,Id,Date,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate,SleepEfficiency
0,1503960366,2016-04-12,327.0,346.0,1985,13162,25,728,NaN,NaN,0.945087
1,1503960366,2016-04-13,384.0,407.0,1797,10735,21,776,NaN,NaN,0.943489
3,1503960366,2016-04-15,412.0,442.0,1745,9762,29,726,NaN,NaN,0.932127
4,1503960366,2016-04-16,340.0,367.0,1863,12669,36,773,NaN,NaN,0.926431
5,1503960366,2016-04-17,700.0,712.0,1728,9705,38,539,NaN,NaN,0.983146
...,...,...,...,...,...,...,...,...,...,...,...
901,8792009665,2016-04-30,343.0,360.0,2896,7174,10,749,NaN,81.464642,0.952778
902,8792009665,2016-05-01,503.0,527.0,1962,1619,0,834,NaN,67.810171,0.954459
903,8792009665,2016-05-02,415.0,423.0,2015,1831,0,916,NaN,67.403919,0.981087
904,8792009665,2016-05-03,516.0,545.0,2297,2421,0,739,NaN,74.371940,0.946789


In [7]:
#forward fill and backward fill BMI 
T1_merged_df.sort_values(by=['Id', 'Date'], inplace=True)
T1_merged_df['BMI_filled'] = T1_merged_df.groupby('Id')['BMI'].ffill()
T1_merged_df['BMI_filled'] = T1_merged_df.groupby('Id')['BMI_filled'].bfill()

null_summary = T1_merged_df.isnull().mean().reset_index()
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T1")
print(null_summary)
T1_merged_df.describe()

T1
                Column  Percent_Null
0                   Id      0.000000
1                 Date      0.000000
2   TotalMinutesAsleep      0.000000
3       TotalTimeInBed      0.000000
4             Calories      0.000000
5           TotalSteps      0.000000
6    VeryActiveMinutes      0.000000
7     SedentaryMinutes      0.000000
8                  BMI     91.525424
9         AvgHeartRate     55.932203
10     SleepEfficiency      0.000000
11          BMI_filled     77.723971


,Id,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate,SleepEfficiency,BMI_filled
count,4.130000e+02,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000,35.000000,182.000000,413.000000,92.000000
mean,5.000979e+09,419.467312,458.639225,2397.573850,8541.140436,25.188862,712.169492,24.828571,73.615234,0.916769,26.233587
std,2.060360e+09,118.344679,127.101607,762.886326,4156.926990,36.388187,165.957432,4.063695,8.295873,0.087039,5.575708
min,1.503960e+09,58.000000,61.000000,257.000000,17.000000,0.000000,0.000000,22.650000,59.377175,0.498361,22.650000
25%,3.977334e+09,361.000000,403.000000,1850.000000,5206.000000,0.000000,631.000000,23.889999,67.049029,0.912181,22.650000
50%,4.702922e+09,433.000000,463.000000,2220.000000,8925.000000,9.000000,717.000000,24.000000,73.043697,0.943128,24.100000
75%,6.962181e+09,490.000000,526.000000,2926.000000,11393.000000,38.000000,783.000000,24.170000,79.368192,0.960688,28.000000
max,8.792010e+09,796.000000,961.000000,4900.000000,22770.000000,210.000000,1265.000000,47.540001,104.871472,1.000000,47.540001


In [8]:
#reduce df to drop rows with threshold
T1_merged_df = T1_merged_df.dropna(thresh=7)

null_summary = T1_merged_df.isnull().mean().reset_index() 
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T1")
print(null_summary)

T1_merged_df.describe()

T1
                Column  Percent_Null
0                   Id      0.000000
1                 Date      0.000000
2   TotalMinutesAsleep      0.000000
3       TotalTimeInBed      0.000000
4             Calories      0.000000
5           TotalSteps      0.000000
6    VeryActiveMinutes      0.000000
7     SedentaryMinutes      0.000000
8                  BMI     91.525424
9         AvgHeartRate     55.932203
10     SleepEfficiency      0.000000
11          BMI_filled     77.723971


,Id,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate,SleepEfficiency,BMI_filled
count,4.130000e+02,413.000000,413.000000,413.000000,413.000000,413.000000,413.000000,35.000000,182.000000,413.000000,92.000000
mean,5.000979e+09,419.467312,458.639225,2397.573850,8541.140436,25.188862,712.169492,24.828571,73.615234,0.916769,26.233587
std,2.060360e+09,118.344679,127.101607,762.886326,4156.926990,36.388187,165.957432,4.063695,8.295873,0.087039,5.575708
min,1.503960e+09,58.000000,61.000000,257.000000,17.000000,0.000000,0.000000,22.650000,59.377175,0.498361,22.650000
25%,3.977334e+09,361.000000,403.000000,1850.000000,5206.000000,0.000000,631.000000,23.889999,67.049029,0.912181,22.650000
50%,4.702922e+09,433.000000,463.000000,2220.000000,8925.000000,9.000000,717.000000,24.000000,73.043697,0.943128,24.100000
75%,6.962181e+09,490.000000,526.000000,2926.000000,11393.000000,38.000000,783.000000,24.170000,79.368192,0.960688,28.000000
max,8.792010e+09,796.000000,961.000000,4900.000000,22770.000000,210.000000,1265.000000,47.540001,104.871472,1.000000,47.540001


In [9]:
#bfill & ffill heartrate
T1_merged_df['AvgHeartRate_filled'] = T1_merged_df.groupby('Id')['AvgHeartRate'].ffill().bfill()

null_summary = T1_merged_df.isnull().mean().reset_index() 
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T1")
print(null_summary)

T1
                 Column  Percent_Null
0                    Id      0.000000
1                  Date      0.000000
2    TotalMinutesAsleep      0.000000
3        TotalTimeInBed      0.000000
4              Calories      0.000000
5            TotalSteps      0.000000
6     VeryActiveMinutes      0.000000
7      SedentaryMinutes      0.000000
8                   BMI     91.525424
9          AvgHeartRate     55.932203
10      SleepEfficiency      0.000000
11           BMI_filled     77.723971
12  AvgHeartRate_filled      0.000000


In [10]:
T1_merged_df.drop('AvgHeartRate', axis=1, inplace=True)
T1_merged_df.drop('BMI', axis=1, inplace=True)
T1_merged_df.rename(columns={'BMI_filled': 'BMI', 'AvgHeartRate_filled': 'AvgHeartRate'}, inplace=True)
T1_merged_df

,Id,Date,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,SleepEfficiency,BMI,AvgHeartRate
0,1503960366,2016-04-12,327.0,346.0,1985,13162,25,728,0.945087,22.65,68.656250
1,1503960366,2016-04-13,384.0,407.0,1797,10735,21,776,0.943489,22.65,68.656250
3,1503960366,2016-04-15,412.0,442.0,1745,9762,29,726,0.932127,22.65,68.656250
4,1503960366,2016-04-16,340.0,367.0,1863,12669,36,773,0.926431,22.65,68.656250
5,1503960366,2016-04-17,700.0,712.0,1728,9705,38,539,0.983146,22.65,68.656250
...,...,...,...,...,...,...,...,...,...,...,...
901,8792009665,2016-04-30,343.0,360.0,2896,7174,10,749,0.952778,NaN,81.464642
902,8792009665,2016-05-01,503.0,527.0,1962,1619,0,834,0.954459,NaN,67.810171
903,8792009665,2016-05-02,415.0,423.0,2015,1831,0,916,0.981087,NaN,67.403919
904,8792009665,2016-05-03,516.0,545.0,2297,2421,0,739,0.946789,NaN,74.371940


GAN Synthesise

In [12]:
def synthesiser(data, id_col, date_col, participant_col, start_participant_num, num_new_participants):
    """
    Generate synthetic data for new participants using CTGAN.

    Parameters:
    - data (pd.DataFrame): Original dataset.
    - id_col (str): Column name to be treated as unique row ID (e.g., 'row_id').
    - date_col (str): Column name representing date for each ID.
    - participant_col (str): Column name representing participant IDs.
    - start_participant_num (int): Number to start naming new participants from (e.g., 17 for 'p17').
    - num_new_participants (int): How many new participants to synthesize.

    Returns:
    - pd.DataFrame: Synthetic data for the new participants.
    """
    
    # Step 0: Assign unique row ID if not already
    if id_col not in data.columns:
        data[id_col] = range(len(data))
    
    # Step 1: Setup metadata
    metadata = SingleTableMetadata()
    metadata.detect_from_dataframe(data)
    metadata.update_column(column_name=id_col, sdtype='id')
    metadata.set_primary_key(column_name=id_col)
    metadata.update_column(column_name=participant_col, sdtype='categorical')
    metadata.update_column(column_name=date_col, sdtype='datetime')

    # Step 2: Fit the synthesizer
    synthesizer = CTGANSynthesizer(metadata)
    synthesizer.fit(data)

    # Step 3: Estimate rows per participant and sample synthetic data
    rows_per_participant = data.groupby(participant_col).size().median()
    rows_per_participant = int(rows_per_participant)
    synthetic_sample_size = rows_per_participant * num_new_participants
    synthetic_data = synthesizer.sample(synthetic_sample_size)

    # Step 4: Generate new participant IDs
    new_ids = [f"p{i}" for i in range(start_participant_num, start_participant_num + num_new_participants)]

    # Step 5: Build new synthetic dataset
    new_participant_data = pd.DataFrame()

    for new_id in new_ids:
        # Sample synthetic data for each new participant
        subset = synthetic_data.sample(rows_per_participant, replace=True).copy()
        subset[participant_col] = new_id
        # Assign new dates for each sampled row
        date_range = pd.date_range(start="2023-01-01", periods=rows_per_participant, freq='D')
        subset[date_col] = date_range
        new_participant_data = pd.concat([new_participant_data, subset], ignore_index=True)

    # Step 6: Reassign row IDs
    new_participant_data[id_col] = range(1, len(new_participant_data) + 1)

    return new_participant_data


start_participant_num = T1_merged_df['Id'].nunique()
T1_synthetic_data = synthesiser(T1_merged_df, id_col='row_id', date_col='Date', participant_col='Id', start_participant_num=start_participant_num, num_new_participants=start_participant_num+100)
T1_synthetic_data.head()

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:1250: UserWarning: No 'datetime_format' is present in the metadata for the following columns:
Column Name   sdtype datetime_format
       Date datetime            None
Without this specification, SDV may not be able to accurately parse the data. We recommend adding datetime formats using 'update_column'.
  warnings.

,Id,Date,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,SleepEfficiency,BMI,AvgHeartRate,row_id
0,p24,2023-01-01,723.0,631.0,1263,8487,0,843,0.865678,NaN,78.080813,1
1,p24,2023-01-02,466.0,436.0,4425,9259,0,1086,0.869761,NaN,76.579460,2
2,p24,2023-01-03,573.0,396.0,2592,12603,0,1209,0.882658,26.15418,59.377175,3
3,p24,2023-01-04,536.0,645.0,3831,4794,0,400,0.976014,NaN,81.446268,4
4,p24,2023-01-05,596.0,197.0,2673,6217,0,935,0.903031,NaN,68.412513,5


In [13]:
T1_final = pd.concat([T1_merged_df, T1_synthetic_data], ignore_index=True)
T1_final.drop('Id', axis=1, inplace=True)
T1_final.drop('Date', axis=1, inplace=True)
T1_final.drop('row_id', axis=1, inplace=True)
T1_final.to_csv("Final_Dataset/sleepDay_merged.csv (derived).csv", index=False)
T1_final.describe()

,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,SleepEfficiency,BMI,AvgHeartRate
count,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,3017.000000,675.000000,3017.000000
mean,499.413656,451.093139,2613.362943,8262.724561,27.385814,857.477627,0.893037,25.474176,73.519385
std,124.976706,165.906063,903.156173,4931.121142,46.523970,225.172138,0.112656,2.767832,8.873398
min,58.000000,61.000000,257.000000,17.000000,0.000000,0.000000,0.498361,22.650000,59.377175
25%,426.000000,359.000000,1938.000000,4456.000000,0.000000,729.000000,0.878043,23.593749,66.904230
50%,514.000000,462.000000,2488.000000,7841.000000,6.000000,853.000000,0.927021,25.917548,71.015505
75%,591.000000,561.000000,3187.000000,11550.000000,32.000000,966.000000,0.959849,26.152482,80.632152
max,796.000000,961.000000,4900.000000,22770.000000,210.000000,1265.000000,1.000000,47.540001,104.871472


In [14]:
T1_final

,TotalMinutesAsleep,TotalTimeInBed,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,SleepEfficiency,BMI,AvgHeartRate
0,327.0,346.0,1985,13162,25,728,0.945087,22.65000,68.656250
1,384.0,407.0,1797,10735,21,776,0.943489,22.65000,68.656250
2,412.0,442.0,1745,9762,29,726,0.932127,22.65000,68.656250
3,340.0,367.0,1863,12669,36,773,0.926431,22.65000,68.656250
4,700.0,712.0,1728,9705,38,539,0.983146,22.65000,68.656250
...,...,...,...,...,...,...,...,...,...
3012,433.0,292.0,2140,11233,29,922,0.997726,NaN,59.377175
3013,170.0,655.0,4311,12117,9,927,0.868230,NaN,69.756636
3014,557.0,516.0,2564,17,47,913,0.498361,NaN,62.480757
3015,476.0,453.0,1750,14219,19,760,0.839257,25.83279,60.718837


# T2

From: dailyActivity_merged.csv, weightLogInfo_merged.csv, sleepDay_merged.csv, heartrate_seconds_merged.csv


Features: TotalSteps, VeryActiveMinutes, LightlyActiveMinutes, SedentaryMinutes, BMI, TotalMinutesAsleep, AvgHeartRate

Label: Calories

In [16]:
T2_sleep = sleepDay_merged[['Id', 'Date', 'TotalMinutesAsleep']]
T2_activity = dailyActivity_merged[['Id', 'Date', 'Calories', 'TotalSteps', 'VeryActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes']]
T2_weight = weightLogInfo_merged[['Id', 'Date', 'BMI']]
T2_heartrate = daily_avg_heartrate[['Id', 'Date', 'AvgHeartRate']]

T2_merged_df = pd.merge(T2_sleep, T2_activity, on=['Id', 'Date'], how='outer')
T2_merged_df = pd.merge(T2_merged_df, T2_weight, on=['Id', 'Date'], how='outer')
T2_merged_df = pd.merge(T2_merged_df, T2_heartrate, on=['Id', 'Date'], how='outer')

T2_merged_df.dropna(how='all', subset=[
    'TotalMinutesAsleep', 'Calories', 'TotalSteps',
    'VeryActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes', 'BMI', 'AvgHeartRate'
], inplace=True)

T2_merged_df

,Id,Date,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate
0,1503960366,2016-04-12,327.0,1985,13162,25,328,728,NaN,NaN
1,1503960366,2016-04-13,384.0,1797,10735,21,217,776,NaN,NaN
2,1503960366,2016-04-14,NaN,1776,10460,30,181,1218,NaN,NaN
3,1503960366,2016-04-15,412.0,1745,9762,29,209,726,NaN,NaN
4,1503960366,2016-04-16,340.0,1863,12669,36,221,773,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
938,8877689391,2016-05-08,NaN,2847,10686,17,245,1174,25.559999,72.550523
939,8877689391,2016-05-09,NaN,3710,20226,73,217,1131,25.610001,89.615738
940,8877689391,2016-05-10,NaN,2832,10733,18,224,1187,NaN,71.544377
941,8877689391,2016-05-11,NaN,3832,21420,88,213,1127,25.559999,89.149122


In [17]:
#forward fill and backward fill BMI 
T2_merged_df.sort_values(by=['Id', 'Date'], inplace=True)
T2_merged_df['BMI_filled'] = T2_merged_df.groupby('Id')['BMI'].ffill()
T2_merged_df['BMI_filled'] = T2_merged_df.groupby('Id')['BMI_filled'].bfill()

#reduce df to drop rows with threshold
T2_merged_df = T2_merged_df.dropna(thresh=8)

#forward fill and backward fill AvgHeartRate 
T2_merged_df['AvgHeartRate_filled'] = T2_merged_df.groupby('Id')['AvgHeartRate'].ffill().bfill()

T2_merged_df.drop('AvgHeartRate', axis=1, inplace=True)
T2_merged_df.drop('BMI', axis=1, inplace=True)
T2_merged_df.rename(columns={'BMI_filled': 'BMI', 'AvgHeartRate_filled': 'AvgHeartRate'}, inplace=True)
T2_merged_df

,Id,Date,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate
0,1503960366,2016-04-12,327.0,1985,13162,25,328,728,22.650000,75.804177
1,1503960366,2016-04-13,384.0,1797,10735,21,217,776,22.650000,75.804177
2,1503960366,2016-04-14,NaN,1776,10460,30,181,1218,22.650000,75.804177
3,1503960366,2016-04-15,412.0,1745,9762,29,209,726,22.650000,75.804177
4,1503960366,2016-04-16,340.0,1863,12669,36,221,773,22.650000,75.804177
...,...,...,...,...,...,...,...,...,...,...
938,8877689391,2016-05-08,NaN,2847,10686,17,245,1174,25.559999,72.550523
939,8877689391,2016-05-09,NaN,3710,20226,73,217,1131,25.610001,89.615738
940,8877689391,2016-05-10,NaN,2832,10733,18,224,1187,25.610001,71.544377
941,8877689391,2016-05-11,NaN,3832,21420,88,213,1127,25.559999,89.149122


In [18]:
null_summary = T2_merged_df.isnull().mean().reset_index() 
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T2")
print(null_summary)

T2
                 Column  Percent_Null
0                    Id      0.000000
1                  Date      0.000000
2    TotalMinutesAsleep     35.062893
3              Calories      0.000000
4            TotalSteps      0.000000
5     VeryActiveMinutes      0.000000
6  LightlyActiveMinutes      0.000000
7      SedentaryMinutes      0.000000
8                   BMI     61.163522
9          AvgHeartRate      0.000000


In [19]:
start_participant_num = T2_merged_df['Id'].nunique()
T2_synthetic_data = synthesiser(T2_merged_df, id_col='row_id', date_col='Date', participant_col='Id', start_participant_num=start_participant_num, num_new_participants=start_participant_num+100)
T2_synthetic_data

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:1250: UserWarning: No 'datetime_format' is present in the metadata for the following columns:
Column Name   sdtype datetime_format
       Date datetime            None
Without this specification, SDV may not be able to accurately parse the data. We recommend adding datetime formats using 'update_column'.
  warnings.

,Id,Date,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate,row_id
0,p27,2023-01-01,408.0,1664,11764,0,121,227,NaN,109.789625,1
1,p27,2023-01-02,NaN,940,9307,32,351,937,28.209216,75.258831,2
2,p27,2023-01-03,NaN,651,10453,19,210,838,NaN,77.808895,3
3,p27,2023-01-04,494.0,918,7147,2,448,1440,47.540001,106.801638,4
4,p27,2023-01-05,475.0,1172,11709,0,383,1113,27.410134,63.826154,5
...,...,...,...,...,...,...,...,...,...,...,...
3551,p153,2023-01-24,322.0,1541,26230,2,295,1077,27.294883,76.389628,3552
3552,p153,2023-01-25,NaN,1363,6362,5,134,1414,26.862210,69.065813,3553
3553,p153,2023-01-26,412.0,1500,9629,0,284,813,21.450001,72.295576,3554
3554,p153,2023-01-27,406.0,2533,25180,0,293,1440,NaN,70.340879,3555


In [20]:
T2_final = pd.concat([T2_merged_df, T2_synthetic_data], ignore_index=True)
T2_final.drop('Id', axis=1, inplace=True)
T2_final.drop('Date', axis=1, inplace=True)
T2_final.drop('row_id', axis=1, inplace=True)
T2_final.to_csv("Final_Dataset/dailyActivity_merged.csv", index=False)
T2_final.describe()

,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate
count,2689.000000,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,1631.000000,4192.000000
mean,414.194868,1953.915792,10620.252385,14.272662,284.122137,959.552004,27.325402,74.792644
std,95.406664,711.974510,6885.197918,26.866662,126.250874,308.738506,4.273453,9.917223
min,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.450001,59.377175
25%,397.000000,1476.000000,5718.000000,0.000000,194.000000,739.000000,26.978067,67.851167
50%,417.000000,1852.000000,9473.500000,1.000000,282.000000,959.500000,27.440356,73.650493
75%,442.000000,2301.000000,14543.000000,15.000000,373.000000,1204.000000,27.972365,79.055392
max,796.000000,4900.000000,29326.000000,210.000000,518.000000,1440.000000,47.540001,109.789625


In [21]:
T2_final

,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate
0,327.0,1985,13162,25,328,728,22.650000,75.804177
1,384.0,1797,10735,21,217,776,22.650000,75.804177
2,NaN,1776,10460,30,181,1218,22.650000,75.804177
3,412.0,1745,9762,29,209,726,22.650000,75.804177
4,340.0,1863,12669,36,221,773,22.650000,75.804177
...,...,...,...,...,...,...,...,...
4187,322.0,1541,26230,2,295,1077,27.294883,76.389628
4188,NaN,1363,6362,5,134,1414,26.862210,69.065813
4189,412.0,1500,9629,0,284,813,21.450001,72.295576
4190,406.0,2533,25180,0,293,1440,NaN,70.340879


# T3

From: dailyActivity_merged.csv, heartrate_seconds_merged.csv, weightLogInfo_merged.csv


Features: TotalSteps, Calories, VeryActiveMinutes, SedentaryMinutes, AvgHeartRate, BMI

Label: TotalMinutesAsleep

In [23]:
T3_sleep = sleepDay_merged[['Id', 'Date', 'TotalMinutesAsleep']]
T3_activity = dailyActivity_merged[['Id', 'Date', 'Calories', 'TotalSteps', 'VeryActiveMinutes', 'SedentaryMinutes']]
T3_weight = weightLogInfo_merged[['Id', 'Date', 'BMI']]
T3_heartrate = daily_avg_heartrate[['Id', 'Date', 'AvgHeartRate']]

T3_merged_df = pd.merge(T3_sleep, T3_activity, on=['Id', 'Date'], how='outer')
T3_merged_df = pd.merge(T3_merged_df, T3_weight, on=['Id', 'Date'], how='outer')
T3_merged_df = pd.merge(T3_merged_df, T3_heartrate, on=['Id', 'Date'], how='outer')

T3_merged_df.dropna(how='all', subset=[
    'TotalMinutesAsleep', 'Calories', 'TotalSteps',
    'VeryActiveMinutes', 'SedentaryMinutes', 'BMI', 'AvgHeartRate'
], inplace=True)

T3_merged_df

,Id,Date,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate
0,1503960366,2016-04-12,327.0,1985,13162,25,728,NaN,NaN
1,1503960366,2016-04-13,384.0,1797,10735,21,776,NaN,NaN
2,1503960366,2016-04-14,NaN,1776,10460,30,1218,NaN,NaN
3,1503960366,2016-04-15,412.0,1745,9762,29,726,NaN,NaN
4,1503960366,2016-04-16,340.0,1863,12669,36,773,NaN,NaN
...,...,...,...,...,...,...,...,...,...
938,8877689391,2016-05-08,NaN,2847,10686,17,1174,25.559999,72.550523
939,8877689391,2016-05-09,NaN,3710,20226,73,1131,25.610001,89.615738
940,8877689391,2016-05-10,NaN,2832,10733,18,1187,NaN,71.544377
941,8877689391,2016-05-11,NaN,3832,21420,88,1127,25.559999,89.149122


In [24]:
# remove all null TotalMinutesAsleep 
T3_merged_df = T3_merged_df[~((T3_merged_df['TotalMinutesAsleep'] == 0) | (T3_merged_df['TotalMinutesAsleep'].isna()))]

#forward fill and backward fill BMI 
T3_merged_df.sort_values(by=['Id', 'Date'], inplace=True)
T3_merged_df['BMI_filled'] = T3_merged_df.groupby('Id')['BMI'].ffill()
T3_merged_df['BMI_filled'] = T3_merged_df.groupby('Id')['BMI_filled'].bfill()

#reduce df to drop rows with threshold
T3_merged_df = T3_merged_df.dropna(thresh=8)

#forward fill and backward fill AvgHeartRate 
T3_merged_df['AvgHeartRate_filled'] = T3_merged_df.groupby('Id')['AvgHeartRate'].ffill().bfill()

T3_merged_df.drop('AvgHeartRate', axis=1, inplace=True)
T3_merged_df.drop('BMI', axis=1, inplace=True)
T3_merged_df.rename(columns={'BMI_filled': 'BMI', 'AvgHeartRate_filled': 'AvgHeartRate'}, inplace=True)
T3_merged_df

,Id,Date,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate
0,1503960366,2016-04-12,327.0,1985,13162,25,728,22.65,68.656250
1,1503960366,2016-04-13,384.0,1797,10735,21,776,22.65,68.656250
3,1503960366,2016-04-15,412.0,1745,9762,29,726,22.65,68.656250
4,1503960366,2016-04-16,340.0,1863,12669,36,773,22.65,68.656250
5,1503960366,2016-04-17,700.0,1728,9705,38,539,22.65,68.656250
...,...,...,...,...,...,...,...,...,...
901,8792009665,2016-04-30,343.0,2896,7174,10,749,NaN,81.464642
902,8792009665,2016-05-01,503.0,1962,1619,0,834,NaN,67.810171
903,8792009665,2016-05-02,415.0,2015,1831,0,916,NaN,67.403919
904,8792009665,2016-05-03,516.0,2297,2421,0,739,NaN,74.371940


In [25]:
null_summary = T3_merged_df.isnull().mean().reset_index() 
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T3")
print(null_summary)

T3
               Column  Percent_Null
0                  Id      0.000000
1                Date      0.000000
2  TotalMinutesAsleep      0.000000
3            Calories      0.000000
4          TotalSteps      0.000000
5   VeryActiveMinutes      0.000000
6    SedentaryMinutes      0.000000
7                 BMI     56.603774
8        AvgHeartRate      0.000000


In [26]:
start_participant_num = T3_merged_df['Id'].nunique()
T3_synthetic_data = synthesiser(T3_merged_df, id_col='row_id', date_col='Date', participant_col='Id', start_participant_num=start_participant_num, num_new_participants=start_participant_num+100)
T3_synthetic_data

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:1250: UserWarning: No 'datetime_format' is present in the metadata for the following columns:
Column Name   sdtype datetime_format
       Date datetime            None
Without this specification, SDV may not be able to accurately parse the data. We recommend adding datetime formats using 'update_column'.
  warnings.

,Id,Date,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate,row_id
0,p14,2023-01-01,58.0,2684,16589,10,1148,22.650000,85.457529,1
1,p14,2023-01-02,355.0,2239,9356,0,751,25.321098,59.377175,2
2,p14,2023-01-03,58.0,2046,156,0,450,25.901283,73.617948,3
3,p14,2023-01-04,58.0,2259,4087,20,730,NaN,61.405403,4
4,p14,2023-01-05,382.0,1843,1081,13,873,NaN,70.782893,5
...,...,...,...,...,...,...,...,...,...,...
1705,p127,2023-01-11,615.0,2400,2046,0,1120,NaN,66.202967,1706
1706,p127,2023-01-12,183.0,962,7624,80,1222,26.703298,62.609757,1707
1707,p127,2023-01-13,287.0,1456,3075,60,1219,NaN,67.252530,1708
1708,p127,2023-01-14,58.0,2096,8746,44,1197,NaN,82.028711,1709


In [27]:
T3_final = pd.concat([T3_merged_df, T3_synthetic_data], ignore_index=True)
T3_final.drop('Id', axis=1, inplace=True)
T3_final.drop('Date', axis=1, inplace=True)
T3_final.drop('row_id', axis=1, inplace=True)
T3_final.to_csv("Final_Dataset/sleepDay_merged.csv", index=False)
T3_final.describe()

,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate
count,1922.000000,1922.000000,1922.000000,1922.000000,1922.000000,904.000000,1922.000000
mean,342.223725,1983.899584,6598.470864,23.993236,805.246618,25.516858,70.066710
std,165.791157,926.723071,4710.991711,26.883140,286.806360,2.793738,7.681127
min,58.000000,403.000000,42.000000,0.000000,2.000000,22.650000,59.377175
25%,238.000000,1438.000000,2650.250000,1.000000,635.000000,24.168389,64.212754
50%,353.000000,1870.500000,6327.500000,16.000000,808.500000,25.611178,69.336111
75%,452.000000,2397.000000,9805.000000,39.750000,1015.000000,26.094202,74.973504
max,775.000000,4900.000000,22770.000000,210.000000,1222.000000,47.540001,104.871472


In [28]:
T3_final

,TotalMinutesAsleep,Calories,TotalSteps,VeryActiveMinutes,SedentaryMinutes,BMI,AvgHeartRate
0,327.0,1985,13162,25,728,22.650000,68.656250
1,384.0,1797,10735,21,776,22.650000,68.656250
2,412.0,1745,9762,29,726,22.650000,68.656250
3,340.0,1863,12669,36,773,22.650000,68.656250
4,700.0,1728,9705,38,539,22.650000,68.656250
...,...,...,...,...,...,...,...
1917,615.0,2400,2046,0,1120,NaN,66.202967
1918,183.0,962,7624,80,1222,26.703298,62.609757
1919,287.0,1456,3075,60,1219,NaN,67.252530
1920,58.0,2096,8746,44,1197,NaN,82.028711


# T4

From: dailyActivity_merged.csv, sleepDay_merged.csv (with computed efficiency)


Features: TotalSteps, Calories, TotalMinutesAsleep, SleepEfficiency

Label: BMI

In [30]:
T4_sleep = sleepDay_merged[['Id', 'Date', 'TotalMinutesAsleep', 'TotalTimeInBed']]
T4_activity = dailyActivity_merged[['Id', 'Date', 'Calories', 'TotalSteps']]
T4_weight = weightLogInfo_merged[['Id', 'Date', 'BMI']]

T4_merged_df = pd.merge(T4_sleep, T4_activity, on=['Id', 'Date'], how='outer')
T4_merged_df = pd.merge(T4_merged_df, T4_weight, on=['Id', 'Date'], how='outer')

T4_merged_df.dropna(how='all', subset=[
    'TotalMinutesAsleep', 'Calories', 'TotalSteps', 'BMI'
], inplace=True)

T4_merged_df['SleepEfficiency'] = T4_merged_df['TotalMinutesAsleep'] / T4_merged_df['TotalTimeInBed']
T4_merged_df['SleepEfficiency'] = T4_merged_df['SleepEfficiency'].replace([np.inf, -np.inf], np.nan)
#T4_merged_df = T4_merged_df[~((T4_merged_df['SleepEfficiency'] == 0) | (T4_merged_df['SleepEfficiency'].isna()))]

T4_merged_df.drop(columns=['TotalTimeInBed'], inplace=True)

T4_merged_df

,Id,Date,TotalMinutesAsleep,Calories,TotalSteps,BMI,SleepEfficiency
0,1503960366,2016-04-12,327.0,1985,13162,NaN,0.945087
1,1503960366,2016-04-13,384.0,1797,10735,NaN,0.943489
2,1503960366,2016-04-14,NaN,1776,10460,NaN,NaN
3,1503960366,2016-04-15,412.0,1745,9762,NaN,0.932127
4,1503960366,2016-04-16,340.0,1863,12669,NaN,0.926431
...,...,...,...,...,...,...,...
938,8877689391,2016-05-08,NaN,2847,10686,25.559999,NaN
939,8877689391,2016-05-09,NaN,3710,20226,25.610001,NaN
940,8877689391,2016-05-10,NaN,2832,10733,NaN,NaN
941,8877689391,2016-05-11,NaN,3832,21420,25.559999,NaN


In [31]:
#forward fill and backward fill BMI 
T4_merged_df.sort_values(by=['Id', 'Date'], inplace=True)
T4_merged_df['BMI_filled'] = T4_merged_df.groupby('Id')['BMI'].ffill()
T4_merged_df['BMI_filled'] = T4_merged_df.groupby('Id')['BMI_filled'].bfill()
T4_merged_df.drop('BMI', axis=1, inplace=True)
T4_merged_df.rename(columns={'BMI_filled': 'BMI'}, inplace=True)

# tbc if want to reduce emptiness in sleep but not sure if makes sense
#T4_merged_df['SleepEfficiency'] = T4_merged_df.groupby('Id')['SleepEfficiency'].ffill()
#T4_merged_df['SleepEfficiency'] = T4_merged_df.groupby('Id')['SleepEfficiency'].bfill()

T4_merged_df = T4_merged_df[~((T4_merged_df['BMI'] == 0) | (T4_merged_df['BMI'].isna()))]

T4_merged_df

,Id,Date,TotalMinutesAsleep,Calories,TotalSteps,SleepEfficiency,BMI
0,1503960366,2016-04-12,327.0,1985,13162,0.945087,22.650000
1,1503960366,2016-04-13,384.0,1797,10735,0.943489,22.650000
2,1503960366,2016-04-14,NaN,1776,10460,NaN,22.650000
3,1503960366,2016-04-15,412.0,1745,9762,0.932127,22.650000
4,1503960366,2016-04-16,340.0,1863,12669,0.926431,22.650000
...,...,...,...,...,...,...,...
938,8877689391,2016-05-08,NaN,2847,10686,NaN,25.559999
939,8877689391,2016-05-09,NaN,3710,20226,NaN,25.610001
940,8877689391,2016-05-10,NaN,2832,10733,NaN,25.610001
941,8877689391,2016-05-11,NaN,3832,21420,NaN,25.559999


In [32]:
null_summary = T4_merged_df.isnull().mean().reset_index() 
null_summary.columns = ['Column', 'Percent_Null']
null_summary['Percent_Null'] = null_summary['Percent_Null'] * 100
print("T4")
print(null_summary)

T4
               Column  Percent_Null
0                  Id      0.000000
1                Date      0.000000
2  TotalMinutesAsleep     52.226721
3            Calories      0.000000
4          TotalSteps      0.000000
5     SleepEfficiency     52.226721
6                 BMI      0.000000


In [ ]:
start_participant_num = T4_merged_df['Id'].nunique()
T4_synthetic_data = synthesiser(T4_merged_df, id_col='row_id', date_col='Date', participant_col='Id', start_participant_num=start_participant_num, num_new_participants=start_participant_num+100)
T4_synthetic_data

C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:778: UserWarning: There is an existing primary key 'row_id'. This key will be removed.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
C:\Users\User\anaconda3\Lib\site-packages\sdv\single_table\base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sdv\metadata\single_table.py:1250: UserWarning: No 'datetime_format' is present in the metadata for the following columns:
Column Name   sdtype datetime_format
       Date datetime            None
Without this specification, SDV may not be able to accurately parse the data. We recommend adding datetime formats using 'update_column'.
  warnings.

In [ ]:
T4_final = pd.concat([T4_merged_df, T4_synthetic_data], ignore_index=True)
T4_final.drop('Id', axis=1, inplace=True)
T4_final.drop('Date', axis=1, inplace=True)
T4_final.drop('row_id', axis=1, inplace=True)
T4_final.to_csv("Final_Dataset/weightLogInfo_merged.csv", index=False)
T4_final.describe()

In [ ]:
T4_final